In [21]:
import numpy as np

In [22]:
#read sample_data_csv
import pandas as pd
file_name = "nudity_train.pkl"
data = pd.read_pickle(f'training_pkl/{file_name}')
data = data.dropna
data

<bound method DataFrame.dropna of         IMDb_id                                                url  None  \
0     tt0111438  https://www.imdb.com/title/tt0111438/parentalg...     2   
1     tt0349080  https://www.imdb.com/title/tt0349080/parentalg...     5   
2     tt0088680  https://www.imdb.com/title/tt0088680/parentalg...    11   
3     tt0066065  https://www.imdb.com/title/tt0066065/parentalg...     3   
4     tt0065481  https://www.imdb.com/title/tt0065481/parentalg...     0   
...         ...                                                ...   ...   
5195  tt5727282  https://www.imdb.com/title/tt5727282/parentalg...  None   
5196  tt0172493  https://www.imdb.com/title/tt0172493/parentalg...  None   
5197  tt0120701  https://www.imdb.com/title/tt0120701/parentalg...  None   
5198  tt1545106  https://www.imdb.com/title/tt1545106/parentalg...  None   
5199  tt0112040  https://www.imdb.com/title/tt0112040/parentalg...  None   

      Mild Moderate Severe  
0       14       20     

In [23]:
data.columns

AttributeError: 'function' object has no attribute 'columns'

In [24]:
#pipeline that would take the data and make two objects, text(the text from column text) and label from column frightening, use word2vec to convert text to vectors, and then use a classifier to predict the label
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [25]:
X = data['IMDb_id']
#y has to be the array from columns 'None', 'Mild', 'Moderate', 'Severe'
y = data[['None', 'Mild', 'Moderate', 'Severe']]
#turn y into a single numpy array where 

TypeError: 'method' object is not subscriptable

In [26]:
import torch.nn as nn
import torch
import numpy as np
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [28]:
X_train = [torch.tensor(tokenizer.encode(text)).float().unsqueeze(0) for text in X_train]
X_test = [torch.tensor(tokenizer.encode(text)).float().unsqueeze(0) for text in X_test]
y_train = [torch.tensor(y).float().unsqueeze(0) for y in y_train.values]
y_test = [torch.tensor(y).float().unsqueeze(0) for y in y_test.values]

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

In [ ]:
X_train[0].size()

In [ ]:
embed_dim = len(X[0])
m = nn.Linear(embed_dim, 5)
input = torch.randn(1, embed_dim)
output = m(input)
print(output.size())

In [ ]:
#logistic regression
class LinearRegression(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearRegression, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)
    def forward(self, x):
        out = self.linear(x)
        return out

In [ ]:
X_train[0].size()

In [29]:
y_train[0].size()

KeyError: 0

In [ ]:
#now build a model where input will be X_train[0] and output will be y_train[0]
model = LinearRegression(6, 4)

In [ ]:
#loss function
criterion = nn.MSELoss()
#optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.000001)

In [ ]:
model.forward(X_train[0])

In [ ]:
#train the model
num_epochs = 100
for epoch in range(num_epochs):
    for i in range(len(X_train)):
        optimizer.zero_grad()
        outputs = model(X_train[i])
        loss = criterion(outputs, y_train[i])
        loss.backward()
        optimizer.step()
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))